In [1]:
# Code were actual at February 2020

In [2]:
import pandas as pd
import requests
import re
from pdb2uniprot import get_uniprot, smiles2pdb

## Import PDB

In [7]:
# smiles of fragment to search in PDB
smiles = 'F'

# make an substructure query in PDB database for wanted smiles
url = "https://www.rcsb.org/pdb/rest/smilesQuery?smiles=" + smiles + "&search_type=substructure"
query = requests.get(url)

# find all PDBs, smiles (InChIKeys) with the smiles and create dataframe
exceptions = []
pdbs = re.findall('structureId="(.{4})"', query.content.decode("utf8"))
smiles = re.findall('\<smiles\>(.*)\<\/smiles\>', query.content.decode("utf8")) 

table = pd.DataFrame(list(zip(pdbs, smiles)), columns = ['PDB_F', 'smiles_F'])

In [8]:
df_result = table.copy()
udict = get_uniprot(list(df_result['PDB_F'].values))

Warning! Uniprots not retrieved for []


In [5]:
# Write to table
for pdb in udict:
    df_result.loc[df_result["PDB_F"] == pdb, 'Uniprot_F'] = udict[pdb]

In [6]:
tab_df = df_result.copy()
df_result2 = pd.DataFrame(columns=tab_df.columns)
for index, row in tab_df.iterrows():
    pdb_Fs = row["Uniprot_F"].split()
    new_row = row.copy()
    for pdb in pdb_Fs:
        new_row["Uniprot_F"] = pdb
        temp_df = pd.DataFrame([list(new_row)], columns=df_result.columns)
        df_result2 = df_result2.append(temp_df, ignore_index=True)
df_result

,PDB_F,smiles_F


In [ ]:
# After F to H change:
pdbs_list = smiles2pdb(smiles["smiles_H"])

In [ ]:
# Write the dictionary value for each pdb to a table
for smi in smiles["smiles_H"]:
    smiles.loc[smiles["smiles_H"] == smi, 'PDB_H'] = pdbs_list[smi]

In [ ]:
tab_F = smiles.copy()
df_result3 = pd.DataFrame(columns=tab_F.columns)
for index, row in tab_F.iterrows():
    pdb_Fs = row["PDB_H"].split()
    new_row = row.copy()
    for pdb in pdb_Fs:
        new_row["PDB_H"] = pdb
        temp_df = pd.DataFrame([list(new_row)], columns=df_result3.columns)
        df_result3 = df_result.append(temp_df, ignore_index=True)
df_result3

In [ ]:
# Get UniProt for H - ligands
df_result4 = df_result3.copy()
udict = get_uniprot(list(df_result4['PDB_H'].values))

In [ ]:
for pdb in udict:
    df_result4.loc[df_result4["PDB_H"] == pdb, 'Uniprot_H'] = udict[pdb]

In [ ]:
tab_F = df_result4.copy()
df_result5 = pd.DataFrame(columns=tab_F.columns)
for index, row in tab_F.iterrows():
    pdb_Fs = row["Uniprot_H"].split()
    new_row = row.copy()
    for pdb in pdb_Fs:
        new_row["Uniprot_H"] = pdb
        temp_df = pd.DataFrame([list(new_row)], columns=df_result.columns)
        df_result5 = df_result5.append(temp_df, ignore_index=True)

df_result.to_csv()

## Compare Uniprots

In [ ]:
table_H = df_result5.copy()
table_F = pd.read_csv()

In [ ]:
# convert table to a dictionary, where each smi_F value corresponds to one or more smi_H
dict_F_H = table_F.groupby('smiles_F')['smiles_H'].apply(set).to_dict()

dict_F_H

In [ ]:
# create a dictionary, where each smiles from table_H corresponds to an array of uniprots
smi_h_udictset = table_H.groupby('smiles_H')['Uniprot_H]'].apply(set).to_dict()
smi_h_udictset

In [ ]:
# create a dictionary, where each smiley from table_F corresponds to an array of uniprots
smi_f_udictset = table_F.groupby('smiles_F')['Uniprot_F'].apply(set).to_dict()
smi_f_udictset

In [ ]:
# create dictionaries, where one pdb corresponds to an array of uniprots
# H - ligands
pdb_h_udictset = table_H.groupby('PDB_H')['Uniprot_H'].apply(set).to_dict()
pdb_h_udictset

# F - ligands
pdb_f_udictset = table_F.groupby('PDB_F')['Uniprot_F'].apply(set).to_dict()
pdb_f_udictset

In [ ]:
# create dictionaries where one smiley corresponds to an array of pdb

# H - ligands
smi2pdb_h_udictset = table_H.groupby('smiles_H')['PDB_H'].apply(set).to_dict()
smi2pdb_h_udictset

# F - ligands
smi2pdb_f_udictset = table_F.groupby('smiles_F')['PDB_F'].apply(set).to_dict()
smi2pdb_f_udictset

In [ ]:
# compare the values of 2 dictionaries and find intersections

c = 0
for smi_f, smi_h_list in dict_F_H.items():

    for smi_h in smi_h_list:

        if smi_h in smi_h_udictset:

            try:
                intersection = smi_f_udictset[smi_f].intersection(smi_h_udictset[smi_h])
            except BaseException:
                intersection = set()

            if intersection:
                c += 1

                pdbs_f = smi2pdb_f_udictset[smi_f]
                pdbs_h = smi2pdb_h_udictset[smi_h]


                for pdb_f in pdbs_f:
                    for pdb_h in pdbs_h:

                        uniprots_f = pdb_f_udictset[pdb_f]
                        uniprots_h = pdb_h_udictset[pdb_h]

                        if intersection.issubset(uniprots_f) == True and intersection.issubset(uniprots_h) == True:

                            print(pdb_f,',', pdb_h, ',', smi_f, ',', smi_h)

## PDB to Ki

In [ ]:
from io import StringIO

In [ ]:
# find experiment data for PDF(F) in MOAD

In [ ]:
def get_binding_activity_df(pdb):

    url = "http://www.bindingmoad.org/files/csv/" + pdb.lower() + ".csv"
    query = requests.get(url)


    if query.status_code == requests.codes.ok:

        string = query.content.decode("utf8")

        string_io = StringIO(string)
        df = pd.read_csv(string_io)

        return df
    else:
        print(f"ligand {pdb} not found in mom database")

In [ ]:
def extract_binding_data(df):

    types = "Unnamed: 4"
    activity = "Unnamed: 6"
    conc_name = "Unnamed: 7"

    data = []
    for i, row in df.iterrows():
        data_dict = {}
        if row["Unnamed: 3"] == "valid":
            a = row[types]
            b = row[activity]
            c = row[conc_name]

            if pd.isna(a) or pd.isna(b) or pd.isna(c):
                print("ligand was found, but data is missing")
            else:
                data_dict.update({"type": a,
                                  "activity": b,
                                  "conc_name": c})
                data.append(data_dict)
                data = pd.DataFrame(data).drop_duplicates().to_dict('records')
    return data

In [ ]:
dfc = table.copy()

In [ ]:
df_new = pd.DataFrame(columns=[*dfc.columns, "type", "activity", 'conc_name'])

for i, row in dfc.iterrows():
    pdb = row['PDB_H']
    actdf = get_binding_activity_df(pdb)
    if actdf is not None:

        actdata = extract_binding_data(actdf)
        for mini_data in actdata:
            row_df = pd.DataFrame([[*row.values, mini_data["type"],
                                                 mini_data["activity"],
                                                 mini_data['conc_name']]], columns=df_new.columns)
            df_new = df_new.append(row_df, ignore_index=True)
df_new

In [ ]:
dfc = table_F.copy()

In [ ]:
df_new = pd.DataFrame(columns=[*dfc.columns, "type", "activity", 'conc_name'])

for i, row in dfc.iterrows():
    pdb = row['PDB_H']
    actdf = get_binding_activity_df(pdb)
    if actdf is not None:

        actdata = extract_binding_data(actdf)
        for mini_data in actdata:
            row_df = pd.DataFrame([[*row.values, mini_data["type"],
                                                 mini_data["activity"],
                                                 mini_data['conc_name']]], columns=df_new.columns)
            df_new = df_new.append(row_df, ignore_index=True)

df_new

In [ ]:
def pdb2pubmed(pdb_list):
    """
    Converts list of pdb to dict {pdb: pubmed} based on PDB bank
    """
    exceptions = []

    result = {}
    for pdb in pdb_list:
        url = "https://www.rcsb.org/pdb/rest/describePDB?structureId=" + pdb
        try:
            pdbdir = requests.get(url)
        except requests.exceptions.RequestException:
            print(f"Exception for {smi} (pdb -> pubmed)")
            exceptions.append(smi)
        pdbsF = re.findall('pubmedId="(.{8})"', pdbdir.content.decode("utf8"))
        pdbsF = list(set(pdbsF))
        string = " ".join(pdbsF)
        print(string)
        result[pdb] = string
    return result

In [ ]:
table2 = table.copy()

table2['PubMedId_H'] = ""
pubmed = pdb2pubmed(table['PDB_F'])
for pdb in pubmed:
    table2.loc[table2["PDB_F"] == pdb, 'PubMedId_F'] = pubmed[pdb]

pubmed = pdb2pubmed(table['PDB_H'])
for pdb in pubmed:
    table2.loc[table2["PDB_H"] == pdb, 'PubMedId_H'] = pubmed[pdb]

In [ ]:
# Compare PubMedId_F & PubMedId_H
table3 = table2.copy()

table3['compare'] = ""
for PubMedId_F in table3['PubMedId_F']:
    table3.loc[table3["PubMedId_F"] == table3["PubMedId_H"], 'compare'] = "same experiment"